### **Wrapper vs. Integration**

| Aspect               | **Wrapper**                             | **Integration**                         |
|----------------------|-----------------------------------------|-----------------------------------------|
| **Definition**       | Simplifies API interaction              | Embeds AI to extend functionality       |
| **Complexity**       | Minimal abstraction over the API        | Deep integration with application logic |
| **System Awareness** | Operate without context of the host system's state or data | Leverage system data (like database schemas) for context-aware responses |
| **Use Cases**        | Sending prompts, receiving responses    | Natural language commands, data import  |
| **Examples**         | PSOpenAI, finetuna, aitoolkit           | dbatools.ai, PSHelp.Copilot, Assistants Chat Extension for GitHub Copilot Chat |

### **Benefits of Integration**

- **Efficiency**: Saves time by automating complex tasks.
- **Accessibility**: Allows non-experts to perform advanced operations using plain language.
- **Intelligence**: Makes informed decisions based on system data.

### **Deep Dive into dbatools.ai as an Actual Integration**

- **Natural Language Processing**:
  - Transforms user queries into actionable SQL or PowerShell commands.
  - Uses OpenAI models to understand intent and context.

- **Function Calling and Structured Output**:
  - Employs OpenAI's advanced features to get structured data, ensuring outputs are in executable formats.

- **System Context Utilization**:
  - Incorporates database schemas and metadata to generate accurate queries.
  - Understands the specific environment (like database instances, tables, columns).

- **Enhanced Workflows**:
  - Automates data import processes from various file types.
  - Provides progress tracking and feedback during operations.

- **Safety Measures**:
  - Checks query safety before execution to prevent harmful operations.
  - Ensures that generated commands are valid and secure.



## Using PSOpenAI to connect to OpenAI

In [ ]:
Request-ChatCompletion -Message "What is a splat?" | Select-Object -ExpandProperty Answer

In [ ]:
Request-ChatCompletion -Message "What is a splat?"

## PowerShell behind the scenes

In [ ]:
# Set the headers
$headers = @{
    "Content-Type"  = "application/json"
    "Authorization" = "Bearer $env:OPENAI_API_KEY"
}

# Prepare the messages array
$msgs = @(
    @{
        role    = "user"
        content = "What is a splat?"
    }
)

# Create the splat for Invoke-RestMethod
$splat = @{
    Uri     = "https://api.openai.com/v1/chat/completions"
    Method  = "Post"
    Headers = $headers
    Body    = @{
        model    = "gpt-4o"
        messages = $msgs
    } | ConvertTo-Json
}

$response = Invoke-RestMethod @splat

# Output the assistant's reply
$response.choices[0].message.content

In [ ]:
$response

### Narrow its context

In [ ]:
# Set default values
$PSDefaultParameterValues["Invoke-RestMethod:Headers"] = $headers
$PSDefaultParameterValues["Invoke-RestMethod:Method"] = "POST"
$PSDefaultParameterValues["Invoke-RestMethod:Uri"] = "https://api.openai.com/v1/chat/completions"
$PSDefaultParameterValues["*:OutVariable"] = "results"

# Prepare the messages array
$msgs = @(
    @{
        role    = "system"
        content = "You are a PowerShell Expert."
    }
    @{
        role    = "user"
        content = "What is a splat?"
    }
)

# Prepare the body
$body = @{
    model    = "gpt-4o"
    messages = $msgs
} | ConvertTo-Json

$response = Invoke-RestMethod -Body $body

# Output the assistant's reply
$response.choices[0].message.content

## How do I know which models I can use?

In [ ]:
(Invoke-RestMethod -Method Get -Uri https://api.openai.com/v1/models).data.id

## The magic 

In [8]:
Import-Module ./dbatools.ai.psd1

$query = "IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[pet]') AND type in (N'U')) DROP TABLE [dbo].[pet] ; IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[pet_vaccinations]') AND type in (N'U')) DROP TABLE [dbo].[pet_vaccinations]"

$splat = @{
    SqlInstance = 'localhost'
    Database    = 'tempdb'
    Query       = $query
}

Invoke-DbaQuery @splat

In [ ]:
$splat = @{
    Path            = './lib/immunization.md'
    JsonSchemaPath  = './lib/immunization.json'
    SqlInstance     = 'localhost'
    Database        = 'tempdb'
    Schema          = 'dbo'
    SystemMessage   = 'Convert text to structured data.'
}

Import-DbaiFile @splat

In [ ]:
Invoke-DbaQuery -SqlInstance localhost -Query "SELECT * FROM tempdb.dbo.pet_vaccinations"